<a href="https://colab.research.google.com/github/Vamsi-Dath/GAN-Evaluation---Brain-MRI/blob/main/GAN_vamsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP (MUST RUN EACH TIME)

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-85827acb-25b2-dde7-f728-264950b53136)


## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Stylegan2-ada-pytorch Prerequisites

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1
!pip install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
     |████████████████████████████████| 17.4 MB 44.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-pytho

Verify the next command results in "1".  If not, go back to the beginning and verify you have a GPU runtime.

NameError: ignored

In [ ]:
torch.cuda.device_count()

NameError: ignored

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 195 kB 9.9 MB/s 
     |████████████████████████████████| 26.9 MB 1.2 MB/s 


## Get the StyleGAN code

In [ ]:
!git clone https://github.com/derekphilipau/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 25.50 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [ ]:
mkdir /content/stylegan2-ada-pytorch/datasets

In [ ]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


# FIRST RUN (RUN ONLY ONCE)

## Create Folders on Your Google Drive

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will create a new folder on your Google Drive, **MachineLearningForArtists**.  Your training data will be stored in **MachineLearningForArtists/MyProject**

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists

In [ ]:
mkdir /content/drive/MyDrive/MachineLearningForArtists/MyProject

## Prepare Your Image Dataset for StyleGAN


### Unzip your dataset

***If your images are not zipped, skip this step.***

If your images are in a .zip file, you will need to unzip them.   Copy the path to your .zip file and paste here under `zip_path`.


In [ ]:
zip_path = '' #@param {type:"string"}

In [ ]:
!unzip "$zip_path"

### Prepare your dataset for StyleGAN


Copy the path to your unzipped dataset folder of images and paste into `dataset_path`

In [ ]:
dataset_path = '/content/drive/MyDrive/resized_dataset' #@param {type:"string"}

The following commands will prepare your dataset.  It will crop all of your images to 512x512 pixels.  ***It will NOT overwrite your original images.***

In [ ]:
!apt install imagemagick webp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-batch inkscape libjxr-tools
  libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fo

In [ ]:
!mkdir /content/stylegan2-ada-pytorch/datasets/raw

In [ ]:
!mkdir /content/stylegan2-ada-pytorch/datasets/processed

In [ ]:
!cp "$dataset_path"/* /content/stylegan2-ada-pytorch/datasets/raw/

In [ ]:
%cd /content/stylegan2-ada-pytorch/datasets/raw/

/content/stylegan2-ada-pytorch/datasets/raw


In [ ]:
!mogrify -type TrueColor -set colorspace sRGB -colorspace sRGB -resize 128x128 -background white -gravity center -extent 128x128 -format jpg -path ../processed *

In [ ]:
%cd /content/stylegan2-ada-pytorch/

/content/stylegan2-ada-pytorch


In [ ]:
!python dataset_tool.py --source=/content/stylegan2-ada-pytorch/datasets/processed/ --dest=./datasets/stylegan_dataset.zip

100% 5124/5124 [00:07<00:00, 670.37it/s]


In [ ]:
!cp ./datasets/stylegan_dataset.zip /content/drive/MyDrive/MachineLearningForArtists/MyProject/

If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Train from Scratch

This cell block will train StyleGAN from scratch.  Training from scratch is much slower than using *transfer learning* on a previously trained model.  However, the purpose of this tutorial is to a) try training for the first time and b) notice how the progress images develop over time.

In [ ]:
!python train.py --outdir=/content/drive/MyDrive/MachineLearningForArtists/MyProject --data=./datasets/stylegan_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./datasets/stylegan_dataset.zip",
    "use_labels": false,
    "max_size": 5124,
    "xflip": true,
    "resolution": 128
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num_fp1

## Go to bed :)

Normal training of a dataset can take days.

Results are stored in your Google Drive under drive/MyDrive/MachineLearningForArtists/MyProject

Each training run is stored in a separate directory.  The initial training run is stored in `00000-stylegan_dataset...`.  If you run training twice, the second run will be stored in `00001-stylegan_dataset...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  In it you should see various images from your original dataset. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.


# RESUME TRAINING

***Make sure you run the above section "SETUP (MUST RUN EACH TIME)" before resuming training.***


In [ ]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


Copy previously saved datset

In [ ]:
!cp /content/drive/MyDrive/MachineLearningForArtists/MyProject/stylegan_dataset.zip ./datasets/

If you had to stop training, the browser window closed, or the Colab session timed out (12-24 hours), you can resume training by using a .pkl file stored on your Google Drive.  For your project, this file will be in `drive/MyDrive/MachineLearningForArtists/MyProject/0000X-stylegan_datset...`

In [ ]:
#@title PKL File Path
#@markdown Copy & Paste .pkl file path here:
pkl_file = '/content/drive/MyDrive/MachineLearningForArtists/MyProject/00001-stylegan_dataset-mirror-paper512-resumecustom/network-snapshot-000080.pkl' #@param {type:"string"}


Run the training, resuming with the last save .pkl file.  Results will be stored in your Google Drive under `MachineLearningForArtists/MyProject`

In [ ]:
cd /content/stylegan2-ada-pytorch/

/content/stylegan2-ada-pytorch


In [ ]:
!python train.py --resume=$pkl_file --outdir=/content/drive/MyDrive/MachineLearningForArtists/MyProject --data=./datasets/stylegan_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./datasets/stylegan_dataset.zip",
    "use_labels": false,
    "max_size": 5124,
    "xflip": true,
    "resolution": 128
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num_fp1

# OPTIONAL: TRANSFER LEARNING

## OR Use Transfer Learning (Optional)

We don't have to train from scratch.  We can train using *transfer learning*.  We will use a StyleGAN2 model already pre-trained on faces to train a new model that is trained against our image dataset.

### Get the faces model

In [ ]:
%cd /content/stylegan2-ada-pytorch

In [ ]:
!apt-get install megatools

In [ ]:
!megadl https://mega.nz/#!eQdHkShY!8wyNKs343L7YUjwXlEg3cWjqK2g2EAIdYz5xbkPy3ng

### Prepare old Faces model for StyleGAN2-ada-pytorch


In [ ]:
!python legacy.py \
    --source=/content/stylegan2-ada-pytorch/ffhq-512-avg-tpurun1.pkl \
    --dest=stylegan2-ada-pytorch-ffhq-512.pkl

### Train using the faces model with your dataset

In [ ]:
!python train.py --resume=/content/stylegan2-ada-pytorch/stylegan2-ada-pytorch-ffhq-512.pkl --outdir=/content/drive/MyDrive/MachineLearningForArtists/MyProject --data=./datasets/stylegan_dataset.zip --gpus=1 --cfg=paper512 --mirror=1 --snap=10 --metrics=none


###GAN EVALUTAION (FID)

In [ ]:
# @title FID For pre-existing Dataset of Alzheimers MRI
# example of calculating the frechet inception distance in Keras for cifar10
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from PIL import Image
import os

# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
images1 =[]
images2 =[]
count = 1
path = "/content/drive/MyDrive/Alzheimer_s Dataset/Alzheimer_s Dataset/"
for file in os.listdir(path):
	image = numpy.array(Image.open(path+file))
	image = resize(image,(128,128))
	if count <= 500 :images1.append(image)
	elif count <=1000:images2.append(image)
	count += 1

images1 = numpy.array(images1)
images2 = numpy.array(images2)

print('Loaded', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

# calculate fid
# fid between images1 and images1
fid = calculate_fid(model, images1, images1)
print('FID (same): %.3f' % fid)
# fid between images1 and images2
fid = calculate_fid(model, images1, images2)
print('FID (different): %.3f' % fid)

Loaded (500, 128, 128) (500, 128, 128)
Scaled (500, 299, 299, 3) (500, 299, 299, 3)
16/16 [==============================] - 130s 8s/step
FID (same): -0.000
16/16 [==============================] - 124s 8s/step
FID (different): 0.039
